In [4]:
import torch
import torchvision

In [5]:
n_epochs = 3
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [6]:
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

 

Extracting /files/MNIST/raw/train-images-idx3-ubyte.gz to /files/MNIST/raw


Extracting /files/MNIST/raw/train-labels-idx1-ubyte.gz to /files/MNIST/raw


Extracting /files/MNIST/raw/t10k-images-idx3-ubyte.gz to /files/MNIST/raw




Extracting /files/MNIST/raw/t10k-labels-idx1-ubyte.gz to /files/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [8]:
test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

In [9]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [10]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

## **SGD Optimizer**



In [11]:
network = Net()
optimizer = optim.SGD(network.parameters(), lr=learning_rate,
                      momentum=momentum)

In [12]:
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

In [13]:
def train(epoch):
  network.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = network(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
    

In [14]:
def test():
  network.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      output = network(data)
      test_loss += F.nll_loss(output, target, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

In [15]:
test()
for epoch in range(1, n_epochs + 1):
  train(epoch)
  test()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 2.3316, Accuracy: 1137/10000 (11%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.339086
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.305619
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.314029
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.260222
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.289430
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.236755
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.186437
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.169749
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.186659
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.034549
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.098917
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.021254
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.800838
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.744249
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.674355
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.547719
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.243401
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.333821
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.086432
Train Epoch: 1 [12160

## **Adadelta Optimizer**

In [17]:
network = Net()
optimizer = torch.optim.Adadelta(network.parameters(), lr=learning_rate)

In [18]:
test()
for epoch in range(1, n_epochs + 1):
  train(epoch)
  test()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 2.3127, Accuracy: 990/10000 (10%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.299444
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.336467
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.313420
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.315544
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.326946
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.306030
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.302570
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.258649
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.321968
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.299757
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.297068
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.315593
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.262308
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.314956
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.272841
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.300663
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.306622
Train Epoch: 1 [10880/60000 (18%)]	Loss: 2.277856
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.269252
Train Epoch: 1 [12160/

## **RMSprop Optimizer**

In [19]:
network = Net()
optimizer =  torch.optim.RMSprop(network.parameters(), lr=learning_rate)

In [20]:
test()
for epoch in range(1, n_epochs + 1):
  train(epoch)
  test()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 2.3183, Accuracy: 1044/10000 (10%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.365749
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.303249
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.242383
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.037002
Train Epoch: 1 [2560/60000 (4%)]	Loss: 1.869596
Train Epoch: 1 [3200/60000 (5%)]	Loss: 1.730382
Train Epoch: 1 [3840/60000 (6%)]	Loss: 1.552168
Train Epoch: 1 [4480/60000 (7%)]	Loss: 1.528336
Train Epoch: 1 [5120/60000 (9%)]	Loss: 1.522800
Train Epoch: 1 [5760/60000 (10%)]	Loss: 1.566123
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.327061
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.487955
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.542668
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.241596
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.358907
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.694281
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.184717
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.230953
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.943334
Train Epoch: 1 [12160

## **Adam Optimizer**

In [21]:
network = Net()
optimizer =  torch.optim.Adam(network.parameters(), lr=learning_rate)

In [22]:
test()
for epoch in range(1, n_epochs + 1):
  train(epoch)
  test()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 2.3194, Accuracy: 1026/10000 (10%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.373675
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.290427
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.260322
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.023844
Train Epoch: 1 [2560/60000 (4%)]	Loss: 1.744058
Train Epoch: 1 [3200/60000 (5%)]	Loss: 1.375185
Train Epoch: 1 [3840/60000 (6%)]	Loss: 1.346870
Train Epoch: 1 [4480/60000 (7%)]	Loss: 1.369603
Train Epoch: 1 [5120/60000 (9%)]	Loss: 1.266460
Train Epoch: 1 [5760/60000 (10%)]	Loss: 1.538399
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.445210
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.252582
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.239792
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.308216
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.337592
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.179425
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.627764
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.104952
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.074283
Train Epoch: 1 [12160